In [ ]:
import os
from os import path
from astropy.time import Time
from astropy.io import fits, ascii
import astropy.units as u
from astropy.table import Table, QTable
from astropy.constants import G

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import h5py
from sqlalchemy import func
from sqlalchemy.orm.exc import NoResultFound
from scipy.optimize import root
from scipy.stats import scoreatpercentile
import tqdm
import schwimmbad
from scipy.optimize import root
import pandas as pd

from thejoker import JokerSamples
from thejoker.sampler import JokerParams, TheJoker
from thejoker.plot import plot_rv_curves

from twoface.config import TWOFACE_CACHE_PATH
from twoface.data import APOGEERVData
from twoface.plot import plot_data_orbits, plot_two_panel
from twoface.mass import get_m2_min, mf
from twoface.ext import FileLoader

from apogeebh.db import db_connect, AllStar, AllVisit, StarResult, Status
from apogeebh.plot import plot_diag_panel

In [ ]:
joker_pars = JokerParams(P_min=1 * u.day,
                         P_max=1024 * u.day,
                         jitter=150 * u.m/u.s)

In [ ]:
sheet = FileLoader.GoogleSheets('1z0ukn8QWJL7dTrPdCFWlhQ6r-X-kwiA4z0YRqalZsY4', 1633658731)
dr14_candidates = sheet.load()

In [ ]:
Session, _ = db_connect('../cache/apogeebh.sqlite')
session = Session()

cache_file = '../cache/apogeebh-joker.hdf5'

In [ ]:
for i in range(5):
    n = session.query(AllStar).join(StarResult).filter(StarResult.status_id == i).count()
    print('status={0}: {1}'.format(i, n))

In [ ]:
m1 = 1.3*u.Msun
m2_cut = 1.4*u.Msun

apogee_ids = []
m2_filter = []
with h5py.File(cache_file, 'r') as f:
    for apogee_id in tqdm.tqdm(f):
        star = AllStar.get_apogee_id(session, apogee_id)
        samples = JokerSamples.from_hdf5(f[apogee_id])
            
        mfs = mf(samples['P'], samples['K'], samples['e'])
        m1s = [m1.value]*len(mfs) * m1.unit
        m2_min = get_m2_min(m1s, mfs)
        m2_mask = np.percentile(m2_min.value, 15)*m2_min.unit > m2_cut

        apogee_ids.append(apogee_id)
        m2_filter.append(m2_mask)

m2_filter = np.array(m2_filter)
apogee_ids = np.array(apogee_ids)

In [ ]:
new_candidates = apogee_ids[(~np.isin(apogee_ids, dr14_candidates['APOGEE_ID'])) & m2_filter]
m2_filter.sum(), len(new_candidates)

In [ ]:
# Third-party
from astropy.utils.misc import isiterable
import astropy.units as u
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import numpy as np

from twoface.mass import get_m2_min, mf
from twoface.plot import plot_data_orbits, plot_phase_fold_residual
from twoface.samples_analysis import MAP_sample

def plot_diag_panel(data, samples, joker_params, M1, data_src=None):
    # Compute things we'll need in plots:
    mfs = mf(samples['P'], samples['K'], samples['e'])
    if not isiterable(M1):
        M1 = [M1.value] * len(mfs) * M1.unit
    M2_mins = get_m2_min(M1, mfs)

    # Now make the plot!
    fig = plt.figure(figsize=(9, 10))
    gs = GridSpec(3, 5)

    ax1 = fig.add_subplot(gs[0, :])
    colormap = dict(DR15='k', LAMOST='tab:orange', Keck='tab:green')
    plot_data_orbits_kw = dict()
    plot_data_orbits_kw.setdefault('xlim_choice', 'tight')
    plot_data_orbits_kw.setdefault('highlight_P_extrema', False)
    plot_data_orbits(data, samples, ax=ax1, n_times=16384,
                     **plot_data_orbits_kw)

    if data_src is not None:
        for src in np.unique(data_src):
            sub_data = data[data_src == src]
            sub_data.plot(ax=ax1, markerfacecolor='none', markeredgewidth=1,
                          markeredgecolor=colormap[src], label=src)
        ax1.legend(loc='best')

    # ---
    sample = MAP_sample(data, samples, joker_params)
    axes2 = [fig.add_subplot(gs[1, 0:3]),
             fig.add_subplot(gs[2, 0:3])]
    plot_phase_fold_residual(data, sample, axes=axes2)
    # plt.setp(axes2[0].get_xticklabels(), visible=False)

    # ---
    ax3 = fig.add_subplot(gs[1, 3:])
    ax3.scatter(samples['P'].to(u.day).value, samples['e'],
                marker='o', linewidth=0, alpha=0.5, s=10)
    ax3.set_xlim(0.8, 2500)
    ax3.set_xscale('log')
    ax3.set_ylim(0, 1)
    ax3.set_ylabel('$e$')
    # plt.setp(ax3.get_xticklabels(), visible=False)

    # ---
    ax4 = fig.add_subplot(gs[2, 3:])
    ax4.scatter(samples['P'].to(u.day).value[:len(M2_mins)], M2_mins.value,
                marker='o', linewidth=0, alpha=0.5, s=10)
    ax4.set_xlim(0.8, 1024)
    ax4.set_xscale('log')
    ax4.set_yscale('log')
    ax4.set_ylim(1E-2, 1E3)
    ax4.axhline(1.4, zorder=-10, color='#cccccc')
    ax4.axhline(M1.value[0], zorder=-1, color='tab:green', 
                alpha=0.5, label='$M_1$', marker='')
    ax4.legend(loc='best')
    ax4.set_ylabel(r'$M_{2,{\rm min}}$' + ' [{0:latex_inline}]'.format(u.Msun))
    ax4.set_xlabel('$P$ [day]')

    for ax in [ax3, ax4]:
        ax.xaxis.set_ticks(10**np.arange(0, 3+0.1, 1))
        ax.yaxis.tick_right()
        ax.yaxis.set_label_position("right")

    return fig

In [ ]:
with h5py.File(cache_file, 'r') as f:
    for apogee_id in tqdm.tqdm(new_candidates):
        star = AllStar.get_apogee_id(session, apogee_id)
        data = star.apogeervdata()
        samples = JokerSamples.from_hdf5(f[apogee_id])
        
        fig = plot_diag_panel(data, samples, M1=m1, 
                              joker_params=joker_pars)
        fig.axes[0].set_title(r'{0} ($\log g = {1}$)'
                              .format(star.apogee_id, star.logg))
        fig.tight_layout()
        fig.savefig('../plots/dr15-candidates/{0}.png'.format(apogee_id), dpi=250)
        plt.close(fig)
        break